In [ ]:
###This script used to clean network flow statistics
###removes local communications (with wifi-routers, hubs etc.), DNS/NTP communications, etc. 
###extracts eTLD+1 or hostnames of remote endpoints
###The output of this script is NOT reproducible since we REMOVED all PII's from the input

In [ ]:
import pandas as pd     
from utils import clean_flow, get_sld

# input file names
file_iot_devices = '../../Device Identification/IoT_devices_v5.csv'
file_flow_stats = "../../Inspector Dataset/New data/flow_stats_with_ip.parquet"
# output file names
file_unique_domains = '../../Endpoint Mapping Data/Domain Data/v5_unique_domains.csv'
file_cleaned_flow = '../../Endpoint Mapping Data/Cleaned Flow/cleaned_flow_stat.csv'

# get iot devices and read flow data
iot_devices = pd.read_csv(file_iot_devices)
flow_stat = pd.read_parquet(file_flow_stats)

# get iot flow
iot_flow = pd.merge(flow_stat, iot_devices, on=['device_id'], how='inner')

# remove Local flows, DNS, NTP, etc. and get eTLD+1 or domain
clean_flow_stat = clean_flow(iot_flow)
hostname_list = clean_flow_stat[['remote_hostname']].drop_duplicates()
hostname_list['domain'] = hostname_list.apply(lambda row: get_sld(row.remote_hostname), axis=1)
clean_flow_stat = pd.merge(clean_flow_stat, hostname_list, on=['remote_hostname'], how='inner')

# exclude domains that is visited by only one device
# excludes 6548 domains communicated by less than 3 devices
device_count = clean_flow_stat.groupby(['domain'])['device_id'].count().sort_values(ascending=False).reset_index()
device_count = device_count[device_count['device_id']>2]
device_count = device_count.rename(columns={"device_id": "device_count"})
clean_flow_stat = pd.merge(clean_flow_stat, device_count, on=['domain'], how='inner')

# excludes devices communicates more than 100 domains
# Only 41 devices excludes from the device list those communicate with more than 100 domains
domain_count = clean_flow_stat.groupby(['device_id'])['domain'].nunique().sort_values(ascending=False).reset_index()
domain_count = domain_count[(domain_count['domain']<=100)]
domain_count = domain_count.rename(columns={"domain": "domain_count"})
clean_flow_stat = pd.merge(clean_flow_stat, domain_count, on=['device_id'], how='inner')

# Find unique domains
domain_list = clean_flow_stat[['domain']].drop_duplicates()


# todo save domain list and flows in file 
# domain_list.to_csv(file_unique_domains, index=False)
# clean_flow_stat.to_csv(file_cleaned_flow, index=False)